In [ ]:
# import libraries
import geopandas as gpd
import pydeck as pdk
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# load parcel data from project folder (Dropbox)
df_parcels = gpd.read_file(os.environ.get('LOCAL_PATH')+"Spatial Data/ny-parcels/NYC_2021_Tax_Parcels_SHP_2203/NewYork_2021_Tax_Parcels_SHP_2203.shp")
# df_buildings = gpd.read_file(os.environ.get('LOCAL_PATH')+"Data/Building Footprints/NYC/geo_export_a80ea1a2-e8e0-4ffd-862c-1199433ac303.shp")

In [ ]:
# show length of dataframe
df_parcels.length

In [ ]:
# subset dataframe

In [ ]:
# set CRS to WGS 84
df_parcels = df_parcels.to_crs(epsg=4326)
# df_buildings = df_buildings.to_crs(epsg=4326)

In [ ]:
# extract lat and lon from the centerpoint
df_parcels['center_point'] = df_parcels['geometry'].centroid
df_parcels['lon'] = df_parcels['center_point'].x
df_parcels['lat'] = df_parcels['center_point'].y

In [ ]:
# view df (optional)
print(df_parcels['geometry'].head())
print(df_parcels['center_point'].head())
print(df_parcels['center_point'].head())
print(df_parcels['lon'].head())
print(df_parcels['lat'].head())

In [ ]:
# display parcels as points
points = pdk.Layer(
    'ScatterplotLayer',
    data=df_parcels,
    get_position='[lon, lat]',
    get_radius=10,
    get_fill_color=[255, 0, 0],
    auto_highlight=True,
    pickable=True
)

# heatmap
heat = pdk.Layer(
    "HeatmapLayer",
    data=df_parcels,
    get_position=["lon", "lat"],
    #aggregation=pdk.types.String("MEAN"),
    aggregation=pdk.types.String("SUM"),
    get_weight=1, # Assuming equal weight for all points
    #threshold=1,
    opacity=0.9,
)

# hexagon
hex = pdk.Layer(
    "HexagonLayer",
    data=df_parcels,
    get_position=["lon", "lat"],
    radius=50,
    opacity=0.9,
    pickable=True
)

# Create the map
view_state = pdk.ViewState(
    latitude=40.76,
    longitude=-74,
    zoom=11,
    max_zoom=20,
    min_zoom=10
)

r = pdk.Deck(
    layers=[hex],
    initial_view_state=view_state
)

# Display the visualization in the notebook (warning: increases notebook size significantly)
# r.show()

# save the visualization to an HTML file (git-ignored)
r.to_html('parcels-map.html')